# Overview of this notebook.

I encountered a potential ibis bug in my production code.  In the process
of trying to replicate it, I came across some unexpected ibis behavior that
is hopefully related to my problem.  This notebook has two sections.  The first
contains the unexpected behavior that I am able to replicate.  The second contains
a stripped down version of the proprietary code I'm working on and cannot share in full.


## Unexpected behavior replicated

In [7]:
# Run imports
import easier as ezr
import ibis
import pandas as pd
from ibis import _

In [8]:
# There should be no tables in the docker pg database table until you run this notebook through once
conn = ibis.postgres.connect(url=ezr.pg_creds_from_env())
print(conn.list_tables())

['one', 'two']


In [9]:
# Create tables I will use for testing
pg = ezr.PG(**ezr.pg_creds_from_env(kind="dict"))
pg.query(
    """
-- Deleting the tables if they already exist
DROP TABLE IF EXISTS one CASCADE;
DROP TABLE IF EXISTS two CASCADE;

-- Creating the 'two' table
CREATE TABLE two (
    id SERIAL PRIMARY KEY,
    time TIMESTAMP WITH TIME ZONE,
    junk INTEGER
);

-- Creating the 'one' table
CREATE TABLE one (
    id SERIAL PRIMARY KEY,
    time TIMESTAMP WITH TIME ZONE,
    two_id INTEGER, 
    junk INTEGER
);

"""
)
pg.run()


# Populate tables with two dataframes
conn = ibis.postgres.connect(url=ezr.pg_creds_from_env())

df_one = pd.DataFrame(
    [
        {
            "id": 1,
            "two_id": 10,
            "time": pd.Timestamp("2023-12-01 00:01:38.966677"),
            "junk": 1,
        },
        {
            "id": 2,
            "two_id": 20,
            "time": pd.Timestamp("2023-12-01 00:02:50.506458"),
            "junk": 1,
        },
    ]
)
df_two = pd.DataFrame(
    [
        {"id": 10, "time": pd.Timestamp("12/10/2023"), "junk": 1},
        {"id": 20, "time": pd.Timestamp("12/20/2023"), "junk": 1},
    ]
)

one = conn.tables.one
two = conn.tables.two


conn.insert("one", df_one)
conn.insert("two", df_two)

In [10]:
# Construct an ibis query
one = conn.tables.one
one = one[
    one.id.isin(
        (
            1,
            2,
        )
    )
]
one = one[["id", "two_id", "time"]].rename({"one_time": "time"})
two = conn.tables.two
two = two[
    [
        "id",
    ]
]
three = one.left_join(two, (one.two_id == two.id))

# Print the sql the query should run
print()
ibis.show_sql(three)
print()

# Actually execute the query.  Everything looks as expected.
three.execute()


WITH t0 AS (
  SELECT
    t4.id AS id,
    t4.time AS time,
    t4.two_id AS two_id,
    t4.junk AS junk
  FROM one AS t4
  WHERE
    t4.id IN (1, 2)
), t1 AS (
  SELECT
    t0.id AS id,
    t0.two_id AS two_id,
    t0.time AT TIME ZONE 'UTC' AS time
  FROM t0
), t3 AS (
  SELECT
    t1.id AS id,
    t1.two_id AS two_id,
    t1.time AT TIME ZONE 'UTC' AS one_time
  FROM t1
), t2 AS (
  SELECT
    t4.id AS id
  FROM two AS t4
)
SELECT
  CAST(t3.id AS INT) AS id,
  t3.two_id,
  t3.one_time AT TIME ZONE 'UTC' AS one_time,
  CAST(t2.id AS INT) AS id_right
FROM t3
LEFT OUTER JOIN t2
  ON t3.two_id = t2.id



,id,two_id,one_time,id_right
0,1,10,2023-12-01 00:01:38.966677+00:00,10
1,2,20,2023-12-01 00:02:50.506458+00:00,20


### This is where the unexpected behavior happens
I do an exact copy/paste of the sql above and create a table from it.
The returned datframe does not have timezone aware timestamps for the `one_time` field. It seems to me like it should.

#### ** Note: It would be really useful to have a method for returning a string of the fully compiled/rendered sql you provide in .show_sql() method.  I'd propose maybe a .get_sql() method, or adding a .show_sql(return_string=True) that only returned the string??


In [11]:
table = conn.sql(
    """
    WITH t0 AS (
      SELECT
        t4.id AS id,
        t4.time AS time,
        t4.two_id AS two_id,
        t4.junk AS junk
      FROM one AS t4
      WHERE
        t4.id IN (1, 2)
    ), t1 AS (
      SELECT
        t0.id AS id,
        t0.two_id AS two_id,
        t0.time AT TIME ZONE 'UTC' AS time
      FROM t0
    ), t3 AS (
      SELECT
        t1.id AS id,
        t1.two_id AS two_id,
        t1.time AT TIME ZONE 'UTC' AS one_time
      FROM t1
    ), t2 AS (
      SELECT
        t4.id AS id
      FROM two AS t4
    )
    SELECT
      CAST(t3.id AS INT) AS id,
      t3.two_id,
      t3.one_time AT TIME ZONE 'UTC' AS one_time,
      CAST(t2.id AS INT) AS id_right
    FROM t3
    LEFT OUTER JOIN t2
      ON t3.two_id = t2.id
    """
)
table.execute()

,id,two_id,one_time,id_right
0,1,10,2023-12-01 00:01:38.966677,10
1,2,20,2023-12-01 00:02:50.506458,20


### Explictly supplying an output schema works

In [6]:
output_schema = ibis.schema(
    {
        "id": "int64",
        "two_id": "int64",
        "one_time": "timestamp('UTC')",
        "id_right": "int64",
    }
)

table = conn.sql(
    """
    WITH t0 AS (
      SELECT
        t4.id AS id,
        t4.time AS time,
        t4.two_id AS two_id,
        t4.junk AS junk
      FROM one AS t4
      WHERE
        t4.id IN (1, 2)
    ), t1 AS (
      SELECT
        t0.id AS id,
        t0.two_id AS two_id,
        t0.time AT TIME ZONE 'UTC' AS time
      FROM t0
    ), t3 AS (
      SELECT
        t1.id AS id,
        t1.two_id AS two_id,
        t1.time AT TIME ZONE 'UTC' AS one_time
      FROM t1
    ), t2 AS (
      SELECT
        t4.id AS id
      FROM two AS t4
    )
    SELECT
      CAST(t3.id AS INT) AS id,
      t3.two_id,
      t3.one_time AT TIME ZONE 'UTC' AS one_time,
      CAST(t2.id AS INT) AS id_right
    FROM t3
    LEFT OUTER JOIN t2
      ON t3.two_id = t2.id
    """,
    schema=output_schema,
)
table.execute()

,id,two_id,one_time,id_right
0,1,10,2023-12-01 00:01:38.966677+00:00,10
1,2,20,2023-12-01 00:02:50.506458+00:00,20


# The actual production bug I can't reproduce
Despite my best efforts I can't reproduce this error on a database that doesn't hold our specific
properity stuff.  :(.  I will paste what I can below and hope that it gives an inkling of a direction to look in.


### This is a stripped down version of the ibis query I'm trying to run
*** Note: This code will not run in this notebook.  It is a modified cut/paste from
my production code. ***

In [ ]:
def doit(conn):
    
    # Query the case table for specific fields
    # and limit to only a few ids for debugging
    case = conn.table("cusomterservicecase")
    case = case[case.id.isin([1, 2, 3])]
    case = case[
        [
            "id",
            "when_started",
            "visit_id",
        ]
    ]

    # My actual production code is running an earlier version
    # of ibis that used relabel.  Switching the mapping order here really created
    # a lot of work for us to upgrade to new verions.  :(
    case = case.rename(
         {v:k for (k, v) in {
            "id": "case_id",
            "when_started": "case_started",
        
        }.items()}
    )
    
    # Generate an expression for a table with information I want joined
    visit conn.table("visit")
    visit = visit[
        [
            "id",
            "submitted_by_id",
        ]
    ]
    visit = visit.rename({v:k for (k, v) in {'id': "visit_id"}.items()})

    # WEIRD THINGS HAPPEND HERE.  If I comment out this join, the "when_started" field
    # is returned with the expected timezone information.  Running this join mysteriously
    # keeps the timezones the same but shifts the underlying time values  by 5 hours.
    # This is the expected offset between UTC and America/New_York timezone.
    # THIS BEHAVIOR DOES NOT HAPPEN IN ibis-frame<7.0.0
    case = case.left_join(visit, 'visit_id')
    
    ibis.show_sql(case)
    display( case.execute())
    
doit()

### Generated SQL
I had to edit this query slightly to remove some confidential info, but here is basically the sql generated by the above query

```sql
WITH t0 AS (
  SELECT
    t5.id AS id,
    t5.when_started AS when_started,
    t5.when_ended AS when_ended,
    t5.when_last_updated AS when_last_updated,
    t5.location AS location,
    t5.visit_id AS visit_id,
    t5.is_read AS is_read,
    t5.when_assigned AS when_assigned,
    t5.priority_id AS priority_id
  FROM cusomterservicecase AS t5
  WHERE
    t5.id IN (1, 2, 3)
), t2 AS (
  SELECT
    t0.id AS id,
    t0.when_started AT TIME ZONE 'UTC' AS when_started,
    t0.visit_id AS visit_id
  FROM t0
), t4 AS (
  SELECT
    t2.id AS case_id,
    t2.when_started AT TIME ZONE 'UTC' AS case_started,
    t2.visit_id AS visit_id
  FROM t2
), t1 AS (
  SELECT
    t5.id AS id,
    t5.submitted_by_id AS submitted_by_id
  FROM visit AS t5
), t3 AS (
  SELECT
    t1.id AS visit_id,
    t1.submitted_by_id AS submitted_by_id
  FROM t1
)
SELECT
  CAST(t4.case_id AS INT) AS case_id,
---------------------------------------------------------------------------------------------
-- I AM VERY SUSPICOUS OF THIS LINE.  NOT CASTING TO TIMEZONESTAMPTZ SEEMS TO DO THE 
-- RIGHT THING??
---------------------------------------------------------------------------------------------
  CAST(t4.case_started AT TIME ZONE 'UTC' AS TIMESTAMPTZ) AS case_started,
---------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------
  t4.visit_id,
  CAST(t3.visit_id AS BIGINT) AS visit_id_right,
  t3.submitted_by_id
FROM t4
LEFT OUTER JOIN t3
  ON t4.visit_id = t3.visit_id

```

## Thoughts
I'm sorry I can't reproduce this error, but I can assure you it's happening, and currently preventing my upgrade to ibis 7.2 in production.  The versions of code I'm using in production are basically those pinned in the requirments file in the project.  This is actually analysis code, so "production" in this case means running on mac os with conda/pip as package manager and connecting to postgresapp running on localhost.  I worry this might be part of the reason I can't reproduce in this container.